Chcemy złączyć wszystkie pliki pathabundance w jeden plik pandasowy, tak aby kolumny do były bakterie a w wierszach znajdywały się nazwa próbki i liczba wystąpień danej bakterii.

In [1]:
import os
import pandas as pd

folder_path = '/home/PaulinaSk/mgr/bubabiba/Filtered'

# pusta lista do trzymania DataFrame'ów
dfs = []

# unikalne ścieżki
all_pathways = set()

# iteracja po wszystkich plikach w folderze
for file in os.listdir(folder_path):
    if file.endswith(".tsv"):
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path, sep='\t', comment='#', header=None, names=["Pathway", "Abundance"])

        # wyciągnięcie nazwy próbki
        with open(file_path, encoding='utf-8') as f:
            for line in f:
                if line.startswith('#'):
                    sample_name = line.strip().split('\t')[1].replace('_Abundance', '')
                    break

        # dodanie ścieżki do zbioru
        all_pathways.update(df["Pathway"].values)

        # tworzy serię: index = pathway, value = abundance
        df_series = df.set_index("Pathway")["Abundance"]
        df_series.name = sample_name
        dfs.append(df_series)

result_df = pd.DataFrame(dfs).fillna(0)

result_df.to_csv("pathways_ab.tsv", sep='\t')


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [4]:
import pandas as pd

result_df = pd.read_csv("pathways_ab.tsv", sep='\t', index_col=0)

# Pierwszy wiersz (wiersz 0) to nagłówki — zapisz je
new_columns = ["NCBI_accession"] + result_df.iloc[0, 1:].tolist()

# Usuń pierwszy wiersz (nagłówki)
result_df = result_df[1:]

# Ustaw nowe kolumny
result_df.columns = new_columns

# Teraz SampleID jest zwykłą kolumną
# Przekształć pozostałe kolumny na wartości liczbowe
for col in result_df.columns[1:]:
    result_df[col] = pd.to_numeric(result_df[col], errors='coerce')

# Sprawdź — SampleID powinno być widoczne
print(result_df.columns[:5])
print(result_df["NCBI_accession"].head())


TypeError: arg must be a list, tuple, 1-d array, or Series

In [ ]:
result_df.to_csv("pathways_ab.tsv", sep='\t')